In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)

In [ ]:
x=mnist.train.images.reshape(55000,28,28)
x=tf.reshape(x,[55000,28,28,1])
y=mnist.train.labels.reshape(55000,10,1)
y=tf.reshape(y,[55000,10,1])
lr=.001
batch_size=50
n_epochs=2

In [ ]:
X=tf.placeholder(tf.float32,[batch_size,784],name="image")
Y=tf.placeholder(tf.float32,[batch_size,10],name="labels")
W=tf.Variable(tf.random_normal(shape=[784,10]),name="weights")
b=tf.Variable(tf.zeros([1,10]),name="bias")

In [ ]:
X_image=tf.reshape(X,[-1,28,28,1])
conv1=tf.layers.conv2d(inputs=X_image,filters=32,kernel_size=[5,5],padding="SAME",activation=tf.nn.relu)
pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
conv2=tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5,5],padding="SAME",activation=tf.nn.relu)
pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
print(pool2.shape)
pool2_flat=tf.reshape(pool2,[-1,7*7*64])
print(pool2_flat.shape)
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
print(dense.shape)
logits = tf.layers.dense(inputs=dense, units=10)
entropy=tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y)
loss=tf.reduce_mean(entropy)
optimizer=tf.train.GradientDescentOptimizer(lr).minimize(loss)

In [ ]:
saver=tf.train.Saver()


In [ ]:
with tf.Session() as sess:
    writer=tf.summary.FileWriter("./graph_log_reg",sess.graph)
    sess.run(tf.global_variables_initializer())
    num_of_batches=int(55000/batch_size)
    for i in range(n_epochs):
        total_loss=0.0
        for j in range(num_of_batches):
            X_batch,Y_batch=mnist.train.next_batch(batch_size)
            j,l=sess.run([optimizer,loss],feed_dict={X:X_batch,Y:Y_batch})
            total_loss+=l
        print(total_loss)
        print("Average loss after %d epoch is %d"%(i+1,float(total_loss)/float(num_of_batches)))
        saved=saver.save(sess,'/home/vishal/cc.ckpt')
writer.close()    
    